In [2]:
import yfinance as yf
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

# pd.set_option("display.max_rows", None)

Scrape ticker symbols of S&P500 stocks

In [3]:
# Set the User-Agent header to a string that mimics a popular web browser to get past firewall rules
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
# Request and parse the web page
r = requests.get("https://www.slickcharts.com/sp500", headers=headers)
soup = BeautifulSoup(r.text, "lxml")
# Extract the text (replacing . with -) from the third cell (Symbol column) of each row of the main table
# (ordered by component weights)
ticker_symbols = [
    tr.find_all("td")[2].text.replace(".", "-")
    for tr in soup.find("tbody").find_all("tr")
]
ticker_symbols[:10]

['AAPL',
 'MSFT',
 'AMZN',
 'NVDA',
 'GOOGL',
 'GOOG',
 'META',
 'BRK-B',
 'TSLA',
 'UNH']

Create database

In [4]:
con = sqlite3.connect("stonks.db")
cur = con.cursor()

In [4]:
def create_tables(drop=False):
    create_tables = """
    CREATE TABLE IF NOT EXISTS exchange (
        id INTEGER PRIMARY KEY NOT NULL,
        name TEXT NOT NULL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS ticker_type (
        id INTEGER PRIMARY KEY NOT NULL,
        name TEXT NOT NULL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS sector (
        id INTEGER PRIMARY KEY NOT NULL,
        name TEXT NOT NULL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS currency (
        id INTEGER PRIMARY KEY NOT NULL,
        iso_code TEXT NOT NULL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS ticker (
        id INTEGER PRIMARY KEY NOT NULL,
        currency_id INTEGER NOT NULL,
        exchange_id INTEGER NOT NULL,
        ticker_type_id INTEGER NOT NULL,
        sector_id INTEGER NOT NULL,
        symbol TEXT NOT NULL UNIQUE,
        name TEXT NOT NULL,
        FOREIGN KEY(currency_id) REFERENCES currency(id),
        FOREIGN KEY(exchange_id) REFERENCES exchange(id),
        FOREIGN KEY(ticker_type_id) REFERENCES ticker_type(id),
        FOREIGN KEY(sector_id) REFERENCES sector(id)
    );
    CREATE TABLE IF NOT EXISTS date (
        id INTEGER PRIMARY KEY NOT NULL,
        date INTEGER NOT NULL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS price (
        ticker_id INTEGER NOT NULL,
        date_id INTEGER NOT NULL,
        open REAL NOT NULL,
        high REAL NOT NULL,
        low REAL NOT NULL,
        close REAL NOT NULL,
        volume INTEGER NOT NULL,
        PRIMARY KEY (ticker_id, date_id),
        FOREIGN KEY(ticker_id) REFERENCES ticker(id),
        FOREIGN KEY(date_id) REFERENCES date(id)
    );
    CREATE INDEX IF NOT EXISTS fk_ticker_idx ON ticker (
        sector_id,
        exchange_id,
        currency_id,
        ticker_type_id
    );
    """
    drop_tables = """
    DROP TABLE IF EXISTS ticker;
    DROP TABLE IF EXISTS exchange;
    DROP TABLE IF EXISTS price;
    DROP TABLE IF EXISTS date;
    DROP TABLE IF EXISTS ticker_type;
    DROP TABLE IF EXISTS sector;
    DROP TABLE IF EXISTS currency;
    """
    if drop:
        cur.executescript(drop_tables)
    cur.executescript(create_tables)
    con.commit()


# Create necessary tables
create_tables(drop=True)

In [5]:
# Define function to check successful index creation
# PKs and unique constrained attributes are automatically indexed
# the price tables composite PK is correctly indexed with a composite index
def list_indexes():
    # Get all tables from sqlite_master
    cur.execute(
        """
        SELECT name
        FROM sqlite_master
        WHERE TYPE = 'table';
        """
    )
    tables = cur.fetchall()
    for table in tables:
        table_name = table[0]
        # Get table info
        cur.execute(f"PRAGMA table_info({table_name});")
        columns = cur.fetchall()
        # Get index info
        cur.execute(f"PRAGMA index_list({table_name});")
        indexes = cur.fetchall()
        print("Table:", table_name)

        # Extract and display info about each individual index
        for index in indexes:
            index_name = index[1]
            unique = index[2]
            index_columns = []

            for column in columns:
                cur.execute(f"PRAGMA index_info({index_name});")
                index_info = cur.fetchall()
                for info in index_info:
                    if info[2] == column[1]:
                        index_columns.append(column[1])
            print("  Index:", index_name)
            print("    Columns:", ", ".join(index_columns))
            print("    Unique:", "Yes" if unique else "No")
        print()


list_indexes()

Table: exchange
  Index: sqlite_autoindex_exchange_1
    Columns: name
    Unique: Yes

Table: ticker_type
  Index: sqlite_autoindex_ticker_type_1
    Columns: name
    Unique: Yes

Table: sector
  Index: sqlite_autoindex_sector_1
    Columns: name
    Unique: Yes

Table: currency
  Index: sqlite_autoindex_currency_1
    Columns: iso_code
    Unique: Yes

Table: ticker
  Index: fk_ticker_idx
    Columns: currency_id, exchange_id, ticker_type_id, sector_id
    Unique: No
  Index: sqlite_autoindex_ticker_1
    Columns: symbol
    Unique: Yes

Table: date
  Index: sqlite_autoindex_date_1
    Columns: date
    Unique: Yes

Table: price
  Index: sqlite_autoindex_price_1
    Columns: ticker_id, date_id
    Unique: Yes



Download market data from Yahoo! Finance's API (experiments)

In [6]:
ticker = yf.Ticker("BRK-B")
info = ticker.info
cols = [
    "symbol",
    "shortName",
    "sector",
    "exchange",
    "quoteType",
    "timeZoneShortName",
    "gmtOffSetMilliseconds",
    "currency",
]

pd.DataFrame(info).iloc[0][cols].to_frame()

,0
symbol,BRK-B
shortName,Berkshire Hathaway Inc. New
sector,Financial Services
exchange,NYQ
quoteType,EQUITY
timeZoneShortName,EDT
gmtOffSetMilliseconds,-14400000
currency,USD


In [7]:
# Initiate tickers instance
tickers = yf.Tickers(" ".join(ticker_symbols))

In [8]:
# Download data and convert index timestamp to unix (seconds) for convenient storage in database
temp = tickers.download(group_by="ticker", start="2023-07-01")
temp.drop(columns=["Dividends", "Stock Splits"], level=1, inplace=True)
temp.index = temp.index.astype("datetime64[s]").astype(int)
[(unix,) for unix in temp.index][:10]

[*********************100%***********************]  504 of 504 completed


[(1688342400,),
 (1688515200,),
 (1688601600,),
 (1688688000,),
 (1688947200,),
 (1689033600,),
 (1689120000,),
 (1689206400,),
 (1689292800,),
 (1689552000,)]

In [9]:
# Verify correct datetime
pd.to_datetime(temp.index, unit="s")

DatetimeIndex(['2023-07-03', '2023-07-05', '2023-07-06', '2023-07-07',
               '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13',
               '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19',
               '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25',
               '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31',
               '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
               '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10',
               '2023-08-11', '2023-08-14', '2023-08-15', '2023-08-16',
               '2023-08-17', '2023-08-18'],
              dtype='datetime64[ns]', name='Date', freq=None)

Download necessary data and fill ticker and it's reference (parent) tables

In [10]:
# Fill database with ticker information
for symbol in ticker_symbols:
    try:
        with con:
            info = tickers.tickers[symbol].info
            con.execute(
                """
                INSERT
                    OR IGNORE INTO currency (iso_code)
                VALUES (:currency)
                """,
                info,
            )
            con.execute(
                """
                INSERT
                    OR IGNORE INTO exchange (name)
                VALUES (:exchange)
                """,
                info,
            )
            con.execute(
                """
                INSERT
                    OR IGNORE INTO ticker_type (name)
                VALUES (:quoteType)
                """,
                info,
            )
            con.execute(
                """
                INSERT
                    OR IGNORE INTO sector (name)
                VALUES (:sector)
                """,
                info,
            )
            con.execute(
                """
                INSERT INTO ticker (
                        name,
                        symbol,
                        currency_id,
                        exchange_id,
                        ticker_type_id,
                        sector_id
                    )
                VALUES (
                        :shortName,
                        :symbol,
                        (
                            SELECT id
                            FROM currency
                            WHERE iso_code = :currency
                        ),
                        (
                            SELECT id
                            FROM exchange
                            WHERE name = :exchange
                        ),
                        (
                            SELECT id
                            FROM ticker_type
                            WHERE name = :quoteType
                        ),
                        (
                            SELECT id
                            FROM sector
                            WHERE name = :sector
                        )
                    )
                """,
                info,
            )
            print(symbol)
    except:
        print(f"failed to insert {symbol}")

AAPL
MSFT
AMZN
NVDA
GOOGL
GOOG
META
BRK-B
TSLA
UNH
JNJ
XOM
JPM
LLY
V
PG
AVGO
HD
MA
CVX
MRK
ABBV
PEP
COST
KO
ADBE
CSCO
WMT
TMO
PFE
MCD
BAC
CRM
ACN
CMCSA
LIN
ABT
NFLX
ORCL
AMD
DHR
WFC
DIS
TXN
PM
failed to insert CAT
COP
AMGN
VZ
UNP
INTU
NEE
INTC
BMY
LOW
NKE
IBM
BA
SPGI
RTX
HON
UPS
QCOM
GE
AMAT
BKNG
DE
MS
SBUX
NOW
PLD
ELV
GS
MDT
ADP
TJX
ISRG
BLK
LMT
T
MDLZ
AXP
GILD
SYK
MMC
ADI
SCHW
VRTX
LRCX
CVS
REGN
ETN
ZTS
CB
C
SLB
AMT
CI
PGR
TMUS
BDX
EOG
MO
FI
SO
BSX
CME
EQIX
DUK
MU
ITW
AON
PYPL
KLAC
SNPS
ATVI
ICE
SHW
APD
PANW
CL
NOC
CSX
FDX
HUM
MPC
CDNS
TGT
WM
ORLY
FCX
MCK
PXD
MMM
HCA
EMR
MAR
PSX
MCO
ROP
USB
PH
CMG
APH
NXPI
GD
PNC
VLO
NSC
AJG
EW
F
MSI
GM
OXY
TT
AZO
ADM
ANET
ECL
SRE
CARR
CHTR
TDG
PCAR
MNST
MCHP
CCI
PSA
ADSK
HES
KMB
NUE
AIG
WMB
CTAS
STZ
AFL
MSCI
DXCM
GIS
WELL
TEL
JCI
AEP
IDXX
COF
D
HLT
MET
IQV
EXC
ROST
ON
PAYX
TFC
O
DOW
TRV
BIIB
EL
SPG
FTNT
CPRT
YUM
DHI
SYY
CTVA
A
ODFL
HAL
DLR
DG
AME
OTIS
MRNA
CTSH
CNC
BKR
AMP
LHX
DD
CEG
KMI
PRU
VRSK
ROK
FIS
CMI
GPN
FAST
PPG
HSY
XEL
BK
GWW
CSGP
DVN
URI


Compare batch download and per ticker data retrieval

In [11]:
%%script echo ≈x3 times faster execution, but higher CPU time ᘇᘏᗢ、
%%time
# Batch retrieve OHLC data
data = tickers.download(start="2022-07-01", end="2023-07-01", group_by="ticker")
data.drop(columns=["Dividends", "Stock Splits"], level=1, inplace=True)
# Convert the date to a unix timestamp (seconds)
data.index = data.index.astype("datetime64[s]").astype(int)
# Inserting date
con.executemany(
    """
INSERT
    OR IGNORE INTO date (date)
VALUES (?)
""",
    [(unix,) for unix in data.index],
)

for symbol in cur.execute(
    """
SELECT symbol
FROM ticker
"""
).fetchall():
    try:
        # Retrieve OHLC data for symbol
        ohlc_data = data[symbol[0]].reset_index()
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient="records")
        with con:
            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(
                f"""
            INSERT INTO price (
                    ticker_id,
                    date_id,
                    OPEN,
                    high,
                    low,
                    close,
                    volume
                )
            VALUES (
                    (
                        SELECT id
                        FROM ticker
                        WHERE symbol = '{symbol[0]}'
                    ),
                    (
                        SELECT id
                        FROM date
                        WHERE date = :Date
                    ),
                    :Open,
                    :High,
                    :Low,
                    :Close,
                    :Volume
                )
            """,
                ohlc_data,
            )
            print(symbol[0])
    except Exception as e:
        print(f"[{symbol[0]}] {e}")

≈x3 times faster execution, but higher CPU time ᘇᘏᗢ、


In [12]:
%%script echo lower CPU time, but ≈x3 times slower execution ᘇᘏᗢ、
%%time
# Per ticker OHLC data retrieval - helps avoid rate limiting
for symbol in cur.execute(
    """
    SELECT symbol
    FROM ticker
    """
).fetchall():
    try:
        # Retrieve OHLC data for symbol
        ohlc_data = tickers.tickers[symbol[0]].history(
            start="2022-07-01", end="2023-07-01"
        )[["Open", "High", "Low", "Close", "Volume"]]
        # Convert the date to a unix timestamp (remove timezone holding local time representations)
        ohlc_data.index = ohlc_data.index.tz_localize(None).astype("int") / 10**9
        ohlc_data.reset_index(inplace=True)
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient="records")
        with con:
            # Inserting date could be optimized
            con.executemany(
                """
                INSERT
                    OR IGNORE INTO date (date)
                VALUES (:Date)
                """,
                ohlc_data,
            )

            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(
                f"""
                INSERT INTO price (
                        ticker_id,
                        date_id,
                        OPEN,
                        high,
                        low,
                        close,
                        volume
                    )
                VALUES (
                        (
                            SELECT id
                            FROM ticker
                            WHERE symbol = '{symbol[0]}'
                        ),
                        (
                            SELECT id
                            FROM date
                            WHERE date = :Date
                        ),
                        :Open,
                        :High,
                        :Low,
                        :Close,
                        :Volume
                    )
            """,
                ohlc_data,
            )
            print(symbol[0])
    except Exception as e:
        print(f"[{symbol[0]}] {e}")

lower CPU time, but ≈x3 times slower execution ᘇᘏᗢ、


Download data and fill date and price tables

In [13]:
# Per ticker OHLC data retrieval - helps avoid rate limiting
for symbol in cur.execute(
    """
    SELECT symbol
    FROM ticker
    """
).fetchall():
    try:
        # Retrieve OHLC data for symbol
        ohlc_data = tickers.tickers[symbol[0]].history(
            start="2022-07-01", end="2023-07-01"
        )[["Open", "High", "Low", "Close", "Volume"]]
        # Convert the date to a unix timestamp (remove timezone holding local time representations)
        ohlc_data.index = ohlc_data.index.tz_localize(None).astype("int") / 10**9
        ohlc_data.reset_index(inplace=True)
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient="records")
        with con:
            # Inserting date could be optimized
            con.executemany(
                """
                INSERT
                    OR IGNORE INTO date (date)
                VALUES (:Date)
                """,
                ohlc_data,
            )

            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter, can be easily fixed if needed
            con.executemany(
                f"""
                INSERT INTO price (
                        ticker_id,
                        date_id,
                        OPEN,
                        high,
                        low,
                        close,
                        volume
                    )
                VALUES (
                        (
                            SELECT id
                            FROM ticker
                            WHERE symbol = '{symbol[0]}'
                        ),
                        (
                            SELECT id
                            FROM date
                            WHERE date = :Date
                        ),
                        :Open,
                        :High,
                        :Low,
                        :Close,
                        :Volume
                    )
            """,
                ohlc_data,
            )
            print(symbol[0])
    except Exception as e:
        print(f"[{symbol[0]}] {e}")

A
AAL
AAP
AAPL
ABBV
ABC
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BG
BIIB
BIO
BK
BKNG
BKR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FLT
FMC
FOX
FOXA
FRT
FSLR
FTNT
FTRE
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN

Create update capabilities

In [14]:
cur.execute(
    """
    SELECT DATE(max(date) + 86400, 'unixepoch')
    FROM price p
        JOIN ticker t ON t.id = p.ticker_id
        JOIN date d ON p.date_id = d.id
    WHERE t.symbol = 'AAPL'
    """
).fetchone()[0]

'2023-07-01'

In [15]:
# Define function for updating ohlc data for a given ticker by it's symbol
def add_new_ohlc(symbol):
    try:
        # Get date for latest entry
        latest_entry = cur.execute(
            """
            SELECT DATE(max(date) + 86400, 'unixepoch')
            FROM price p
                JOIN ticker t ON t.id = p.ticker_id
                JOIN date d ON p.date_id = d.id
            WHERE t.symbol = ?
            """,
            (symbol,),
        ).fetchone()[0]
        # Retrieve new OHLC data for symbol
        ohlc_data = tickers.tickers[symbol].history(start=latest_entry)[
            ["Open", "High", "Low", "Close", "Volume"]
        ]
        # Convert the date to a unix timestamp (remove timezone holding local time representations)
        ohlc_data.index = ohlc_data.index.tz_localize(None).astype("int") / 10**9
        ohlc_data.reset_index(inplace=True)
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient="records")
        with con:
            # Inserting date could be optimized
            con.executemany(
                """
                INSERT
                    OR IGNORE INTO date (date)
                VALUES (:Date)
                """,
                ohlc_data,
            )

            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(
                f"""
                INSERT INTO price (
                        ticker_id,
                        date_id,
                        OPEN,
                        high,
                        low,
                        close,
                        volume
                    )
                VALUES (
                        (
                            SELECT id
                            FROM ticker
                            WHERE symbol = '{symbol}'
                        ),
                        (
                            SELECT id
                            FROM date
                            WHERE date = :Date
                        ),
                        :Open,
                        :High,
                        :Low,
                        :Close,
                        :Volume
                    )
                """,
                ohlc_data,
            )
            print(symbol)
    except Exception as e:
        print(f"[{symbol}] {e}")


add_new_ohlc("AAPL")

AAPL


In [16]:
# Check number of entries per ticket name to confirm that update works
cur.execute(
    """
    SELECT COUNT(),
        name
    FROM price
        JOIN ticker ON ticker.id = price.ticker_id
    GROUP BY ticker.name
    """
).fetchall()

[(251, '3M Company'),
 (251, 'A.O. Smith Corporation'),
 (251, 'AFLAC Incorporated'),
 (251, 'AMETEK, Inc.'),
 (251, 'ANSYS, Inc.'),
 (251, 'APA Corporation'),
 (251, 'AT&T Inc.'),
 (251, 'AbbVie Inc.'),
 (251, 'Abbott Laboratories'),
 (251, 'Accenture plc'),
 (251, 'Activision Blizzard, Inc'),
 (251, 'Adobe Inc.'),
 (251, 'Advance Auto Parts Inc.'),
 (251, 'Advanced Micro Devices, Inc.'),
 (251, 'Agilent Technologies, Inc.'),
 (251, 'Air Products and Chemicals, Inc'),
 (251, 'Akamai Technologies, Inc.'),
 (251, 'Alaska Air Group, Inc.'),
 (251, 'Albemarle Corporation'),
 (251, 'Alexandria Real Estate Equities'),
 (251, 'Align Technology, Inc.'),
 (251, 'Allegion plc'),
 (251, 'Alliant Energy Corporation'),
 (251, 'Allstate Corporation (The)'),
 (502, 'Alphabet Inc.'),
 (251, 'Altria Group, Inc.'),
 (251, 'Amazon.com, Inc.'),
 (251, 'Amcor plc'),
 (251, 'Ameren Corporation'),
 (251, 'American Airlines Group, Inc.'),
 (251, 'American Electric Power Company'),
 (251, 'American Express Co

In [17]:
# List all dates
cur.execute(
    """
    SELECT DATE(date, 'unixepoch')
    FROM date
    """
).fetchall()

[('2022-07-01',),
 ('2022-07-05',),
 ('2022-07-06',),
 ('2022-07-07',),
 ('2022-07-08',),
 ('2022-07-11',),
 ('2022-07-12',),
 ('2022-07-13',),
 ('2022-07-14',),
 ('2022-07-15',),
 ('2022-07-18',),
 ('2022-07-19',),
 ('2022-07-20',),
 ('2022-07-21',),
 ('2022-07-22',),
 ('2022-07-25',),
 ('2022-07-26',),
 ('2022-07-27',),
 ('2022-07-28',),
 ('2022-07-29',),
 ('2022-08-01',),
 ('2022-08-02',),
 ('2022-08-03',),
 ('2022-08-04',),
 ('2022-08-05',),
 ('2022-08-08',),
 ('2022-08-09',),
 ('2022-08-10',),
 ('2022-08-11',),
 ('2022-08-12',),
 ('2022-08-15',),
 ('2022-08-16',),
 ('2022-08-17',),
 ('2022-08-18',),
 ('2022-08-19',),
 ('2022-08-22',),
 ('2022-08-23',),
 ('2022-08-24',),
 ('2022-08-25',),
 ('2022-08-26',),
 ('2022-08-29',),
 ('2022-08-30',),
 ('2022-08-31',),
 ('2022-09-01',),
 ('2022-09-02',),
 ('2022-09-06',),
 ('2022-09-07',),
 ('2022-09-08',),
 ('2022-09-09',),
 ('2022-09-12',),
 ('2022-09-13',),
 ('2022-09-14',),
 ('2022-09-15',),
 ('2022-09-16',),
 ('2022-09-19',),
 ('2022-09

In [18]:
main_query = """
SELECT t.symbol,
    t.name,
    s.name,
    DATE(d.date, 'unixepoch'),
    p.open,
    p.high,
    p.low,
    p.close,
    p.volume,
    e.name,
    tt.name,
    c.iso_code
FROM price p
    JOIN ticker t ON p.ticker_id = t.id
    JOIN date d ON p.date_id = d.id
    JOIN sector s ON t.sector_id = s.id
    JOIN exchange e ON t.exchange_id = e.id
    JOIN currency c ON t.currency_id = c.id
    JOIN ticker_type tt ON t.ticker_type_id = tt.id
"""

In [19]:
cur.execute(main_query).fetchmany(3)

[('A',
  'Agilent Technologies, Inc.',
  'Healthcare',
  '2022-07-01',
  117.99027717779944,
  118.95374810979271,
  115.91434363539602,
  118.4074478149414,
  1119800,
  'NYQ',
  'EQUITY',
  'USD'),
 ('A',
  'Agilent Technologies, Inc.',
  'Healthcare',
  '2022-07-05',
  116.95727967718092,
  119.86755454042228,
  115.70576016770362,
  119.74835968017578,
  1484900,
  'NYQ',
  'EQUITY',
  'USD'),
 ('A',
  'Agilent Technologies, Inc.',
  'Healthcare',
  '2022-07-06',
  119.98675012194727,
  121.51637582513143,
  119.74836040002126,
  120.36418914794922,
  1642300,
  'NYQ',
  'EQUITY',
  'USD')]

In [20]:
con.commit()
con.close()